In [1]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, DataCollatorWithPadding,default_data_collator
from torch.utils.data import DataLoader

In [2]:

from datasets import Dataset, Value, ClassLabel, Features
features = Features({"text": Value("string"),"label":ClassLabel(num_classes = 29,names=['盜竊罪', '爆竊罪', '猥褻侵犯罪', '侵害人身罪', '不小心駕駛罪', '販運危險藥物罪', '入屋犯法罪', 
            '搶劫罪', '以欺騙手段取得財產罪', '販毒罪', '販運毒品罪', '普通襲擊罪', '洗黑錢罪', '串謀詐騙罪', 
            '處理贓物罪', '刑事恐嚇罪', '串謀勒索罪', '非法入境罪', '強姦罪', '非禮罪', '縱火罪', '扒竊罪', 
            '管有危險藥物罪', '危險駕駛引致他人死亡罪', '襲擊致造成身體傷害罪', '管有虛假文書罪', '使用虛假文書罪',
             '危險駕駛引致他人身體受嚴重傷害罪', '危險駕駛罪']) ,"begin": Value("string")})
dataset = load_dataset("json", data_files="/home/huijie/legal/crime_prediction/data_prepare/crime_prediction.json",features=features)
dataset = dataset["train"]
dataset.features


Using custom data configuration default-511fb7c719a2c1cb
Reusing dataset json (/home/huijie/.cache/huggingface/datasets/json/default-511fb7c719a2c1cb/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/1 [00:00<?, ?it/s]

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=29, names=['盜竊罪', '爆竊罪', '猥褻侵犯罪', '侵害人身罪', '不小心駕駛罪', '販運危險藥物罪', '入屋犯法罪', '搶劫罪', '以欺騙手段取得財產罪', '販毒罪', '販運毒品罪', '普通襲擊罪', '洗黑錢罪', '串謀詐騙罪', '處理贓物罪', '刑事恐嚇罪', '串謀勒索罪', '非法入境罪', '強姦罪', '非禮罪', '縱火罪', '扒竊罪', '管有危險藥物罪', '危險駕駛引致他人死亡罪', '襲擊致造成身體傷害罪', '管有虛假文書罪', '使用虛假文書罪', '危險駕駛引致他人身體受嚴重傷害罪', '危險駕駛罪'], id=None),
 'begin': Value(dtype='string', id=None)}

In [15]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
from transformers import AutoModel
model = AutoModel.from_pretrained("xlm-roberta-base")

model.eval()

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


XLMRobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(250002, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): 

In [4]:
labels = ['盜竊罪', '爆竊罪', '猥褻侵犯罪', '侵害人身罪', '不小心駕駛罪', '販運危險藥物罪', '入屋犯法罪', 
            '搶劫罪', '以欺騙手段取得財產罪', '販毒罪', '販運毒品罪', '普通襲擊罪', '洗黑錢罪', '串謀詐騙罪', 
            '處理贓物罪', '刑事恐嚇罪', '串謀勒索罪', '非法入境罪', '強姦罪', '非禮罪', '縱火罪', '扒竊罪', 
            '管有危險藥物罪', '危險駕駛引致他人死亡罪', '襲擊致造成身體傷害罪', '管有虛假文書罪', '使用虛假文書罪',
             '危險駕駛引致他人身體受嚴重傷害罪', '危險駕駛罪']
label2id =  {k:v for v,k in enumerate(labels)}
dataset = dataset.align_labels_with_mapping(label2id, "label")


Loading cached processed dataset at /home/huijie/.cache/huggingface/datasets/json/default-511fb7c719a2c1cb/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-6849bb085e66de49.arrow


In [5]:
dataset[:2]

{'text': ['方第一證人需要透過手機作出紀錄。本席裁定控方第一證人當時正在依法執行公務。  陳大律師陳述，控方未能在毫無合理疑點下，證明被告的行為構成阻礙。  高等法院原訟法庭暫委法官黃崇厚（當時官階），在香港特別行政區訴尹明義 案指出，《簡易程序治罪條例》第23條和《侵害人身罪條例》第36（b）條的罪行不同，因為《簡易程序治罪條例》的條文中，沒有故意（wilful）這字眼，所以控方毋須證明被告人故意阻礙執行公務，意即毋須證明阻礙執行公務是被告人進行某行為的意圖（intention）。以《簡易程序治罪條例》第23條這罪行來說，控方',
  ' 沒有任何法例賦予被告權力，容許他當時自行蒐證或執法，因此，縱使他真誠相信他人違反私穩法例，亦不可以强搶他人物品。立法會議員並沒此方面的權力。本席找不到任何法理支持被告搶手機的行為。被告的行為肯定是非法的武力。  基於前述，控方已在毫無合理疑點下，證明所有普通襲擊罪的控罪元素。本席裁定此項控罪罪名成立。 阻礙公職人員執行公務  根據《釋義及通則條例》第3條，公職人員（public officer）指任何在特區政府擔任受薪職位的人。控方第一證人是保安局高級行政主任，必然是公職人員。本席接納案發當日該小組負責在立法會大樓'],
 'label': [3, 11],
 'begin': ['許智峯是立法會議員，可以進入位於中環立法會道1號的立法會綜合大樓，履行公務。 2018年4月24日，立法會大樓會議室1內正舉行《廣深港高鐵條例草案》委員會會議。 當日，政府人員組成一個小組，成員包括保安局高級行政主任梁諾施，及六名運輸及房屋局的人員。他們在立法會大樓不同位置候命，並記錄立法會議員在草案委員會會議舉行時的行蹤。運輸及房屋局助理秘書長運輸3B黎惠珊，是該小組的組長，負責監察及統籌該小組組員；運輸及房屋局助理秘書長運輸3C鄭朗峰，是該小組統籌員；運輸及房屋局二級系統分析／程序編製主任運輸1梁榮燊，',
  '許智峯是立法會議員，可以進入位於中環立法會道1號的立法會綜合大樓，履行公務。 2018年4月24日，立法會大樓會議室1內正舉行《廣深港高鐵條例草案》委員會會議。 當日，政府人員組成一個小組，成員包括保安局高級行政主任梁諾施，及六名運輸及房屋局的人員。他們在立法會大樓不同位置候命，並記錄立法會議員在草案委員會會議舉行時的行蹤。運

In [9]:

def tokenize_function(examples):
    result = tokenizer([examples["begin"],examples["text"]],truncation = True,max_length = 512)
    # if tokenizer.is_fast:
    #     result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    result["input_ids"] = result["input_ids"][0]+ result["input_ids"][1]
    result["attention_mask"] = result["attention_mask"][0]+ result["attention_mask"][1]
    return result

tokenized_datasets = dataset.map(
        tokenize_function, batched=False, remove_columns=["begin","text"],num_proc =16
    )






In [35]:
set(tokenized_datasets['label'][:200])
# tokenized_datasets.map()

{0,
 2,
 3,
 4,
 5,
 6,
 7,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 27,
 28}

In [11]:
# label2id = {'B': 0, 'C': 1}
# tokenized_datasets.align_labels_with_mapping(label2id, "a_similar_to")
# tokenized_datasets = tokenized_datasets.rename_column("label","classification")
data_collator = DataCollatorWithPadding(tokenizer,padding= "max_length",max_length = 512,pad_to_multiple_of= 8)
# train_dataloader = DataLoader(tokenized_datasets,8,collate_fn=data_collator)
train_dataloader = DataLoader(tokenized_datasets,8,collate_fn=data_collator)

In [ ]:
def focal_loss(inputs: Tensor, targets: Tensor,device,
                alpha: float = 4.0,beta: float= .0,reduction: str='mean',
                eval: bool = False) -> Tensor:
    """
    inputs: A float tensor of arbitrary shape.
                The predictions for each example.
    targets: A float tensor with the same shape as inputs. Stores the binary
                classification label for each element in inputs
            (-1 for the negative class and 1 for the positive class).
    """

    prediction = None
    if eval:
        prediction = (torch.sigmoid(inputs.mul(alpha).add(beta)) > 0.5).float()
    print(inputs.shape,targets.shape)
    loss = F.logsigmoid(inputs.mul(targets).mul(alpha).add(beta))
    loss = loss.sum(dim = -1).div(-alpha) # b 
    if reduction == 'mean':
        loss = loss.mean()
    elif reduction == 'sum':
        loss = loss.sum()
    return loss, prediction

In [36]:
import torch
from torch.nn import functional as F 
focal_loss = torch.hub.load(
	'adeelh/pytorch-multi-class-focal-loss',
	model='FocalLoss',
	# alpha=torch.tensor([.75, .25]),
	gamma=2,
	reduction='mean',
	force_reload=False
)
y = F.one_hot(label,num_classes=29)
print(y)
x, y = torch.randn(8, 29), label
print(x.shape,y.shape)
loss = focal_loss(x, y)
print(x,y,loss)
torch.argmax(F.softmax(x,dim =-1),dim =-1 )

tensor([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0]])
torch.Size([8, 29]) torch.Size([8])
tensor([[-6.4583e-01, -3.9616e-01,  9.1491e-01, -3.9861e-01,  6.7614e-01,
          1.3132e+00,  2.7350e+00, -6.184

Using cache found in /home/huijie/.cache/torch/hub/adeelh_pytorch-multi-class-focal-loss_master


tensor([ 6,  4,  1, 15,  0, 22, 23, 23])

In [18]:
for batch in train_dataloader:
    label = batch.pop("labels")
    for key,item in batch.items():
        print(key+": " ,item.shape)
    out = model(**batch)
    print(out)
    break

input_ids:  torch.Size([8, 512])
attention_mask:  torch.Size([8, 512])
BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.0807,  0.1494,  0.0778,  ..., -0.1175,  0.1036,  0.0192],
         [-0.0732,  0.0051, -0.0360,  ...,  0.2372,  0.0604, -0.0624],
         [ 0.0651,  0.1328, -0.0253,  ..., -0.1565, -0.0925,  0.0021],
         ...,
         [ 0.0718,  0.1522,  0.0205,  ..., -0.2209,  0.0358,  0.0645],
         [ 0.0718,  0.1522,  0.0205,  ..., -0.2209,  0.0358,  0.0645],
         [ 0.0718,  0.1522,  0.0205,  ..., -0.2209,  0.0358,  0.0645]],

        [[ 0.0847,  0.1457,  0.0725,  ..., -0.1094,  0.0993,  0.0049],
         [-0.0308, -0.0061, -0.0442,  ...,  0.2323,  0.0551, -0.0845],
         [ 0.0589,  0.1275, -0.0280,  ..., -0.1733, -0.0806, -0.0188],
         ...,
         [ 0.0773,  0.1486,  0.0145,  ..., -0.2129,  0.0300,  0.0485],
         [ 0.0773,  0.1486,  0.0145,  ..., -0.2129,  0.0300,  0.0485],
         [ 0.0773,  0.1486,  0.0145,  ..., -0.2129,  0.

In [49]:
eval_dataloader = DataLoader(vaild,batch_size=4,collate_fn=con_fun) 
eval_dataloader

In [50]:
from tqdm import tqdm

for batch in eval_dataloader:
    a = batch.pop("classification")
    print(a)
    print(batch)
    break
    

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length.

In [ ]:
print( precision    recall  f1-score   support\n\n           0       0.48      1.00      0.65        16\n           1       0.00      0.00      0.00         1\n           2       0.00      0.00      0.00         1\n           3       0.00      0.00      0.00         3\n           4       0.33      1.00      0.50         2\n           5       0.54      0.88      0.67         8\n           6       0.50      1.00      0.67         8\n           7       0.00      0.00      0.00         6\n           8       0.00      0.00      0.00         1\n           9       0.00      0.00      0.00         2\n          11       0.00      0.00      0.00         3\n          12       0.00      0.00      0.00         3\n          13       0.00      0.00      0.00         1\n          14       0.00      0.00      0.00         1\n          20       0.00      0.00      0.00         1\n          21       0.00      0.00      0.00         2\n          22       0.00      0.00      0.00         4\n          26       0.00      0.00      0.00         1\n          28       0.00      0.00      0.00         4\n\n    accuracy                           0.49        68\n   macro avg       0.10      0.20      0.13        68\nweighted avg       0.25      0.49      0.33        68\n)